In [1]:
import sys, os
from pathlib import Path

# notebook lives in /notebooks, code lives in ../src
repo_root = Path().resolve().parent
sys.path.insert(0, str(repo_root / "src"))


import json
import pandas as pd
from datasets import Dataset
from itertools import product
from tqdm.auto import tqdm

from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision

from search import SurfSpotRetriever
from forecast import get_weekend_forecast
from report_generator import SurfReportGenerator

with open("C:/Users/laech/OneDrive/Documents/surfreporter/data/ground_truth_rag_surf.json", "r", encoding="utf-8") as f:
    all_gt = json.load(f)

forecast = get_weekend_forecast()

generation_models = ["gpt-4o", "o3-mini"]
temperatures      = [0.3, 0.7]
top_ks            = [3, 10]

all_summaries = []

for gen_model, temperature, top_k in tqdm(
    product(generation_models, temperatures, top_ks),
    total=len(generation_models)*len(temperatures)*len(top_ks),
    desc="Param configs"
):
    
    retriever = SurfSpotRetriever()
    generator = SurfReportGenerator(
        spots=[],  # placeholder, we'll fill per‐query
        forecast=forecast,
        generation_model=gen_model,
        temperature=temperature
    )

    
    rows = []
    for item in tqdm(all_gt, desc="Queries", leave=False):
        q  = item["Query"]
        gt = item["Expected Answer"]
        
        spots = retriever.retrieve_spots(
            user_query=q,
            preferred_direction=item["Direction of Wave"],
            preferred_bottom=item["Type of Bottom"],
            top_k=top_k
        )
        contexts = [s["description"] for s in spots]
        
        answer = generator.generate_report(q)

        rows.append({
            "question":           q,
            "ground_truth":       gt,
            "answer":             answer,
            "retrieved_contexts": contexts
        })

    # Build a HuggingFace Dataset for RAGAS
    df = pd.DataFrame(rows)
    ds = Dataset.from_pandas(df)

    # RAGAS evaluation on the three selected metrics
    results = evaluate(
        ds,
        metrics=[faithfulness, answer_relevancy, context_precision],
        show_progress=False  
    )

    # Aggregate each metric’s mean score
    summary = {
        "model":            gen_model,
        "temperature":      temperature,
        "top_k":            top_k,
        "faithfulness":     sum(results["faithfulness"]) / len(results["faithfulness"]),
        "answer_relevancy": sum(results["answer_relevancy"]) / len(results["answer_relevancy"]),
        "context_precision":sum(results["context_precision"]) / len(results["context_precision"])
    }
    all_summaries.append(summary)


results_df = pd.DataFrame(all_summaries)


print("## Final Evaluation Summary")
print(results_df)


c:\Users\laech\OneDrive\Documents\surfreporter\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Param configs: 100%|██████████| 8/8 [1:55:33<00:00, 866.70s/it]

## Final Evaluation Summary
     model  temperature  top_k  faithfulness  answer_relevancy  \
0   gpt-4o          0.3      3      0.277848          0.847651   
1   gpt-4o          0.3     10      0.282392          0.832735   
2   gpt-4o          0.7      3      0.226368          0.850636   
3   gpt-4o          0.7     10      0.288837          0.859793   
4  o3-mini          0.3      3      0.385298          0.727977   
5  o3-mini          0.3     10      0.302095          0.771550   
6  o3-mini          0.7      3      0.395244          0.794630   
7  o3-mini          0.7     10      0.379595          0.790656   

   context_precision  
0           0.456215  
1           0.448682  
2           0.450565  
3           0.447269  
4           0.450565  
5           0.443974  
6           0.453390  
7           0.447269  


In [2]:
results_df.to_pickle("full_eval_summary_openai.pkl")